<h1>Example: Convolutional Neural Network using TensorFlow (2.3.0) </h1>

<h3>Convolution Neural Network</h3>

![CNN](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/CNN.png)

<h3>MNIST Dataset</h3>

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. 

![MNIST](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/MnistExamples.png)

<h4>Step 1: Import Libraries</h4>

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import pandas as pd

print(tf.__version__)

2.3.0


<h4>Step 2: Parameters</h4>

In [2]:
# MNSIT dataset parameters (No need of num_input and dropout as in version 1.15)
num_classes=10

# Training parameters

learning_rate=0.001
num_steps=500
batch_size=128
display_step=10

# Network parameters (No tf graph input through placeholder as in version 1.15)

conv1_filters=32 # first layer
conv2_filters=64 # second layer
fc1_units=1024 #neurons in fully connected layer

<h3> Step4: MNIST dataset from Keras dataset </h3>

<h4> In version 1.15, we used tensorflow.examples.tutorials.mnist packaage, which is no more available in version 2, so it is better to use datasets from keras </h4>

In [3]:
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test)= mnist.load_data() # get training and testing data

#x is the data for images and y is the classes and labels

x_train,x_test = np.array(x_train,np.float32),np.array(x_test,np.float32) #First convert the data to float32
x_train,x_test = x_train/255,x_test/255 # Normalize the image value from [0,255] to [0,1]

In [4]:
# Shuffle and Batch Data
# -- In version 1.15, we used batch size in sess.run nad used mnist.train.next_batch(batch_size) but in 2.3.0 we will
# use tf.data for training data

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

<h3> Step 5: Model creation, Loss Function, Accuracy Metric, Optimizer </h3>

<h4> Version 1.15 vs 2.3.0: </h4>

<h4> 1. For class definition, the process is same but in version 1.15, weights and biases are used to define layers, flatten,fully connected layer, dropout and output whereas in 2.3.0, number of filters and kernel size is used. In dropout now, rate is used instead of keep_prob from previous version.</h4>

<h4> 2. In loss function, previous version used 'softmax_cross_entropy_with_logits', which has been replaced from nn Module package function 'sparse_softmax_cross_entropy_with_logits'.</h4>

<h4> 3. In optimizer, there is no change </h4>

In [5]:
class conv_total(Model):
    
    def __init__(self):
        super(conv_total,self).__init__() #allow us to create objects from nn module without initializing them explicitly
        
        self.conv1=layers.Conv2D(32,kernel_size=5,activation=tf.nn.relu) #1st Convolution Layer [32 filters, kernal =5]
        self.maxpool1=layers.MaxPool2D(2,strides=2) # down-sampling with kernal size 2 and strides of 2
        
        self.conv2=layers.Conv2D(64,kernel_size=3,activation=tf.nn.relu) #1st Convolution Layer [64 filters, kernal =3]
        self.maxpool2=layers.MaxPool2D(2,strides=2) # down-sampling with kernal size 2 and strides of 2
        
        self.flatten=layers.Flatten() # Flatten data to 1D vector for fully connected layer
        
        self.fc1=layers.Dense(1024) #Fully connected layer with 1024 neurons
        
        self.dropout=layers.Dropout(rate=0.5) #Dropout only for true training else not applied
        
        self.out=layers.Dense(num_classes) #output layer (classes/labels)
    
    def call(self,x,is_training=False): #For prediction
        x=tf.reshape(x,[-1,28,28,1])
        x=self.conv1(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.maxpool2(x)
        x=self.flatten(x)
        x=self.fc1(x)
        x=self.dropout(x,training=is_training)
        x=self.out(x)
        
        if not is_training:
            x=tf.nn.softmax(x) #apply softmax when not training
        return x

## Build Neural Network
        
conv_total=conv_total()              
        

In [6]:
# Loss function

def cross_entropy_loss(x,y):
    y=tf.cast(y,tf.int64)  # convert labels to int64
    loss=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=x) #softmax to logits and compute cross entropy
    
    return tf.reduce_mean(loss)

In [7]:
#  Accuracy

def accuracy(y_pred,y_true):
    correct_prediction=tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),axis=-1)

In [8]:
# Optimizer

optimizer = tf.optimizers.Adam(learning_rate)

<h3> Step 6: Optimization process </h3>

<h4> In this version we wrap computation using GradientTape and update the variables and compute gradients </h4>


In [9]:
def optimization(x,y):
    with tf.GradientTape() as g: # computing optimization process as session run
        pred=conv_total(x,is_training=True) #using prediction funtion in conv_total()
        loss=cross_entropy_loss(pred,y) #compute loss
        
    train_var=conv_total.trainable_variables # variables update
    grad=g.gradient(loss,train_var) #compute gradients
    
    optimizer.apply_gradients(zip(grad,train_var)) #update weights and biases following gradients
    

<h3> Step 7: Train Model </h3>

<h4> In this version, we use .take instead of .next_batch</h4>


In [10]:
for step, (batch_x,batch_y) in enumerate(train_data.take(num_steps),1):
    optimization(batch_x,batch_y)
    
    if step % display_step==0:
        pred=conv_total(batch_x)
        loss=cross_entropy_loss(pred,batch_y)
        acc=accuracy(pred,batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

# Test model on validation set (accuracy)
pred = conv_total(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

step: 10, loss: 1.901503, accuracy: 0.757812
step: 20, loss: 1.604883, accuracy: 0.898438
step: 30, loss: 1.579184, accuracy: 0.929688
step: 40, loss: 1.619967, accuracy: 0.890625
step: 50, loss: 1.567867, accuracy: 0.953125
step: 60, loss: 1.523846, accuracy: 0.968750
step: 70, loss: 1.513477, accuracy: 0.968750
step: 80, loss: 1.535113, accuracy: 0.945312
step: 90, loss: 1.540618, accuracy: 0.945312
step: 100, loss: 1.528899, accuracy: 0.968750
step: 110, loss: 1.508346, accuracy: 0.984375
step: 120, loss: 1.499631, accuracy: 0.992188
step: 130, loss: 1.524543, accuracy: 0.960938
step: 140, loss: 1.490476, accuracy: 1.000000
step: 150, loss: 1.498449, accuracy: 0.976562
step: 160, loss: 1.482981, accuracy: 0.992188
step: 170, loss: 1.499913, accuracy: 0.984375
step: 180, loss: 1.491768, accuracy: 0.984375
step: 190, loss: 1.506130, accuracy: 0.976562
step: 200, loss: 1.497489, accuracy: 0.976562
step: 210, loss: 1.486004, accuracy: 0.992188
step: 220, loss: 1.499008, accuracy: 0.9687